# Wrapping a template library

A template library is a library where there are only template classes that can be instantiated.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider various *C++* template classes instantiations during the `Parse` step.
It is therefore required to install the `pyclanglite` `parser`.

The **Standard Template Library (STL)** library is a *C++* library that provides a set of common *C++* template classes such as containers and associative arrays.
These classes can be used with any built-in or user-defined type that supports some elementary operations (e.g. copying, assignment).
It is divided in four components called algorithms, containers, functional and iterators.
**STL** containers (e.g. `std::vector`, `std::set`) are used in many *C++* libraries.
In such a case, it does not seem relevant that every wrapped *C++* library contains wrappers for usual **STL** containers (e.g. `std::vector< double >`, `std::set< int >`).
We therefore proposed *Python* bindings for sequence containers (i.e. `pair`, `array`, `vector`, `deque`, `forward_list` and `list` of the `std` namespace) and associative containers (`set`, `multiset`, `map`, `multimap`, `unordered_set`, `unordered_multiset`, `unordered_map` and `unordered_multimap` of the `std` namespace).
These template instantiations are done for *C++* fundamental types (`bool`, `signed char`, `unsigned char`, `char`, `wchar_t`, `int` (with sign modifiers `signed` and `signed` combined or not with size modifiers `short`, `long` and `long long`), `float`, `double`, `long double`) and strings (`string`, `wstring` of the `std` namespace).
For ordered associative containers both `std::less` and `std::greater` comparators are used.
We here only illustrate the procedure on the `std::vector` template class.
For the complete procedure refers to the `AutoWIG.py` file situed at the root of the **PySTL** [repository](https://github.com/StatisKit/PySTL).

In [1]:
!git clone https://github.com/StatisKit/STL STL
!git -C STL checkout b9569c67ebc59482dc99a8fa11aa685faebc981d

Cloning into 'STL'...
remote: Counting objects: 655, done.
remote: Total 655 (delta 0), reused 0 (delta 0), pack-reused 655
Receiving objects: 100% (655/655), 403.41 KiB | 0 bytes/s, done.
Resolving deltas: 100% (388/388), done.
Note: checking out 'b9569c67ebc59482dc99a8fa11aa685faebc981d'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at b9569c6... Merge pull request #20 from pfernique/master


Then, to install and compile the *C++* library we use available **Conda** recipes.

In [2]:
!conda build -q STL/conda/libstatiskit_stl -c statiskit
!conda install -y -q libstatiskit_stl --use-local -c statiskit

BUILD START: libstatiskit_stl-1.0.0-0

The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2l             |                0         3.2 MB
    path.py-10.3.1             |           py27_0          48 KB
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

    libdev:       1.0.0-py27_0  statiskit
    openssl:      1.0.2l-0               
    path.py:      10.3.1-py27_0          
    pip:          9.0.1-py27_1           
    python:       2.7.13-0               
    python-scons: 2.5.0-py27_0  statiskit
    pyyaml:       3.12-py27_0            
    readline:     6.2-2                  
    setuptools:   27.2.0-py27_0          
    sqlite:       3.13.0-0               
    tk:           8.5.18-0               
    wheel:        0.29.0-py27_0          
    y

As presented below, in order to wrap a template library, the user needs to write headers containing aliases for desired template class instantiations.

In [3]:
!pygmentize STL/src/cpp/STL.h

#ifndef STATISKIT_STL_H
#define STATISKIT_STL_H

#include <vector>
#include <set>
#include <unordered_set>
#include <string>

#if defined WIN32 || defined _WIN32 || defined __CYGWIN__
    #ifdef LIBSTATISKIT_STL
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllexport))
        #else
            #define STATISKIT_STL_API __declspec(dllexport)
        #endif
    #else
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllimport))
        #else
            #define STATISKIT_STL_API __declspec(dllimport)
        #endif
    #endif
#else
    #if __GNUC__ >= 4
        #define STATISKIT_STL_API __attribute__ ((visibility ("default")))
    #else
        #define STATISKIT_STL_API
    #endif
#endif

namespace statiskit
{
    typedef unsigned long int Index;
    typedef std::set< Index > Indices;

    namespace stl
    {

        template<class T>
            class Generator
            {
                public:
                    Generat

Once these preliminaries done, we can proceed to the actual generation of wrappers for the **PySTL** library.
For this, we import **AutoWIG** and create an empty Abstract Semantic Graph (ASG).

We need then to install the *C++* headers. This is done using the `cpp` target in **SCons**.

In [4]:
!scons cpp -C STL

scons: Entering directory `/home/main/AutoWIG/doc/examples/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/src/cpp/STL.h" as "/home/main/miniconda/include/statiskit/stl/STL.h"
g++ -o build/src/cpp/STL.os -c -x c++ -std=c++11 -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fdiagnostics-color=always -fPIC -DLIBSTATISKIT_STL -I/home/main/miniconda/include build/src/cpp/STL.cpp
g++ -o /home/main/miniconda/lib/libstatiskit_stl.so -shared build/src/cpp/STL.os -L/home/main/miniconda/lib
scons: done building targets.


Once the headers habe been installed in the system, we parse headers with relevant compilation flags.

In [5]:
!scons autowig -c -C STL
!scons autowig -C STL

scons: Entering directory `/home/main/AutoWIG/doc/examples/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Cleaning targets ...
Removed /home/main/miniconda/include/statiskit/stl/STL.h
scons: done cleaning targets.
scons: Entering directory `/home/main/AutoWIG/doc/examples/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/src/cpp/STL.h" as "/home/main/miniconda/include/statiskit/stl/STL.h"
Install file: "build/src/cpp/AutoWIG.py" as "/home/main/miniconda/lib/python2.7/site-packages/autowig/site/controller/statiskit_stl.py"
autowig: Generating Boost.Python interface ...
/home/main/miniconda/lib/python2.7/site-packages/clanglite/autowig_parser.py:860: Warning: 
  warnings.warn('', Warning)
scons: done building targets.


Here is an example of the generated wrappers.
We here present the wrappers for the `std::vector< int >` class.

In [6]:
!pygmentize STL/src/py/wrapper/wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp

#include "_stl.h"



namespace autowig
{

    void method_decorator_bb1e0852f2ca56c094260a03787426c7(class ::std::vector< int, class ::std::allocator< int > > & instance, ::std::vector< int, class ::std::allocator< int > >::size_type  param_in_0, int param_out) { instance.at(param_in_0) = param_out; }
    void method_decorator_7ec1ac72b0b05f3a9707175bcd5da0bd(class ::std::vector< int, class ::std::allocator< int > > & instance, int param_out) { instance.front() = param_out; }
    void method_decorator_ed1cf37568ed54cbbd326e6ccbe5f27d(class ::std::vector< int, class ::std::allocator< int > > & instance, int param_out) { instance.back() = param_out; }
}

#if defined(_MSC_VER)
    #if (_MSC_VER == 1900)
namespace boost
{
    template <> class ::std::vector< int, class ::std::allocator< int > > const volatile * get_pointer<class ::std::vector< int, class ::std::allocator< int > > const volatile >(class ::std::vector< int, class ::std::allocator< int > > const volatile *c) { return c; }
}
 

    class_6b9ae5eac40858c9a0f5e6e21c15d1d3.def("front", autowig::method_decorator_7ec1ac72b0b05f3a9707175bcd5da0bd);
    class_6b9ae5eac40858c9a0f5e6e21c15d1d3.def("front", method_pointer_b7cadb076a605b51b2601b9b3480c6b5, boost::python::return_value_policy< boost::python::return_by_value >(), "");
    class_6b9ae5eac40858c9a0f5e6e21c15d1d3.def("back", method_pointer_ed1cf37568ed54cbbd326e6ccbe5f27d, boost::python::return_value_policy< boost::python::return_by_value >(), "");
    class_6b9ae5eac40858c9a0f5e6e21c15d1d3.def("back", autowig::method_decorator_ed1cf37568ed54cbbd326e6ccbe5f27d);
    class_6b9ae5eac40858c9a0f5e6e21c15d1d3.def("back", method_pointer_aaa6ab4cb09b56e3adee1ae72ce60d90, boost::python::return_value_policy< boost::python::return_by_value >(), "");
    class_6b9ae5eac40858c9a0f5e6e21c15d1d3.def("push_back", method_pointer_341df5e2719858f39df095cac9121eaf, "");
    class_6b9ae5eac40858c9a0f5e6e21c15d1d3.def("pop_back", method_pointer_bbaecaa6c9535f04a1ffda1223792c23, "

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [7]:
!conda build STL/conda/python-statiskit_stl -c statiskit 
!conda install -y python-statiskit_stl --use-local -c statiskit --force

BUILD START: python-statiskit_stl-3.3.1-py27_0
    (actual version deferred until further download or env creation)

The following NEW packages will be INSTALLED:

    coverage:         4.3.4-py27_0           
    icu:              54.1-0                 
    libboost:         1.61.0-py27_0 statiskit
    libdev:           1.0.0-py27_0  statiskit
    librun:           1.0.0-py27_0  statiskit
    libstatiskit_stl: 1.0.0-0       local    
    nose:             1.3.7-py27_1           
    openssl:          1.0.2l-0               
    path.py:          10.3.1-py27_0          
    pip:              9.0.1-py27_1           
    python:           2.7.13-0               
    python-dev:       1.0.0-py27_0  statiskit
    python-scons:     2.5.0-py27_0  statiskit
    pyyaml:           3.12-py27_0            
    readline:         6.2-2                  
    setuptools:       27.2.0-py27_0          
    sqlite:           3.13.0-0               
    tk:               8.5.18-0               
    whee

In file included from /home/main/miniconda/conda-bld/python-statiskit_stl_1496215549755/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl/include/boost/python/converter/arg_to_python.hpp:10:0,
                 from /home/main/miniconda/conda-bld/python-statiskit_stl_1496215549755/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl/include/boost/python/call.hpp:15,
                 from /home/main/miniconda/conda-bld/python-statiskit_stl_1496215549755/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl/include/boost/python/object_core.hpp:14,
                 from /home/main/min

In file included from /home/main/miniconda/conda-bld/python-statiskit_stl_1496215549755/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl/include/boost/python.hpp:41:0,
                 from build/src/py/wrapper/_stl.h:4:
/home/main/miniconda/conda-bld/python-statiskit_stl_1496215549755/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl/include/boost/python/make_constructor.hpp: In member function 'void boost::python::detail::install_holder<T>::dispatch(U*, mpl_::true_) const':
/home/main/miniconda/conda-bld/python-statiskit_stl_1496215549755/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place

g++ -o build/src/py/wrapper/wrapper_dc5522842bc75d8b9ac7b46020c60854.os -c -x c++ -std=c++11 -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -I/home/main/miniconda/conda-bld/python-statiskit_stl_1496215549755/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl/include -I/home/main/miniconda/conda-bld/python-statiskit_stl_1496215549755/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl/include/python2.7 build/src/py/wrapper/wrapper_dc5522842bc75d8b9ac7b46020c60854.cpp
g++ -o build/src/py/wrapper/wrapper_d48105936d4f5d09a78d2aa9f878fdb8.os -c -x c++ -std=c++11 -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_N

g++ -o build/src/py/wrapper/wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.os -c -x c++ -std=c++11 -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -I/home/main/miniconda/conda-bld/python-statiskit_stl_1496215549755/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl/include -I/home/main/miniconda/conda-bld/python-statiskit_stl_1496215549755/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl/include/python2.7 build/src/py/wrapper/wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp
g++ -o build/src/py/wrapper/wrapper_6436891c9b6854f494789a812891cbe5.os -c -x c++ -std=c++11 -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_N

+ source /home/main/miniconda/bin/activate /home/main/miniconda/conda-bld/python-statiskit_stl_1496215549755/_t_env
+ /home/main/miniconda/conda-bld/python-statiskit_stl_1496215549755/_t_env/bin/python -s /home/main/miniconda/conda-bld/python-statiskit_stl_1496215549755/test_tmp/run_test.py
import: u'statiskit.stl'
+ /bin/bash -x -e /home/main/miniconda/conda-bld/python-statiskit_stl_1496215549755/test_tmp/run_test.sh
+ nosetests test -x -s -v -A 'level <= 3 and linux'
Test set initialization ... ok
Test set manipulation ... ok
Test set latex representation ... ok
Test set string representation ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.007s

OK
TEST END: /home/main/miniconda/conda-bld/linux-64/python-statiskit_stl-3.3.1-py27_0.tar.bz2
INFO:conda_build.config:--dirty flag not specified.  Removing build folder after successful build/test.

# If you want to upload package(s) to anaconda.org later, type:

anaconda upload /home/main/mini

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [8]:
from statiskit.stl import VectorInt
v = VectorInt()
v.push_back(-1)
v.push_back(0)
v.push_back(1)
v

(-1, 0, 1)

In [9]:
list(v)

[-1, 0, 1]

In [10]:
v[0]

-1

In [11]:
v[0] = -2
v[0]

-2

In [12]:
VectorInt([0, 1])

(0, 1)